In [ ]:
# Cell 1: Mount and Authenticationfrom google.colab import drive, authdrive.mount('/content/drive')auth.authenticate_user()print("✅ Drive mounted and authenticated")

In [ ]:
# Cell 2: Dependencies and Setupimport os, json, time, tracebackfrom datetime import datetimefrom googleapiclient.discovery import buildfrom googleapiclient.http import MediaFileUploadimport ioimport tempfiledrive_service = build('drive', 'v3')FOLDER_ID = "1tzHn4J3QntSLJlJNXcNJe3cLdILGEb3Z"print(f"✅ Drive service ready")print(f"📁 Folder: {FOLDER_ID}")

In [ ]:
# Cell 3: Test Folder Accesstry:    query = f"'{FOLDER_ID}' in parents and trashed=false"    results = drive_service.files().list(q=query, fields="files(id, name)").execute()    files = results.get('files', [])    print(f"✅ Folder accessible, {len(files)} files found")    if files:        print("📁 Sample files:")        for f in files[:3]:            print(f"   - {f['name']}")except Exception as e:    print(f"❌ Folder access error: {e}")    print("Check folder ID and permissions")

In [ ]:
# Cell 4: Processor Class (TESTED)class WorkingProcessor:    def __init__(self):        self.processed = set()        self.running = False        self.stats = {'processed': 0, 'errors': 0}        print("✅ Processor initialized")            def get_commands(self):        try:            query = f"'{FOLDER_ID}' in parents and name contains 'command_' and trashed=false"            results = drive_service.files().list(q=query, fields="files(id, name)").execute()            commands = [f for f in results.get('files', []) if f['id'] not in self.processed]            if commands:                print(f"📨 Found {len(commands)} new commands")            return commands        except Exception as e:            print(f"❌ Get commands error: {e}")            return []        def execute_code(self, code):        import sys        from io import StringIO                old_stdout = sys.stdout        sys.stdout = StringIO()                try:            # Safe namespace with common imports            namespace = {                '__name__': '__main__',                'print': print,                'datetime': datetime,                'time': time,                'os': os,                'json': json            }                        # Add numpy, pandas if available            try:                import numpy as np                import pandas as pd                namespace['np'] = np                namespace['pd'] = pd            except ImportError:                pass                        exec(code, namespace)            output = sys.stdout.getvalue()                        return {                'status': 'success',                'output': output,                'timestamp': time.time()            }        except Exception as e:            return {                'status': 'error',                 'error': str(e),                'traceback': traceback.format_exc(),                'timestamp': time.time()            }        finally:            sys.stdout = old_stdout        def write_response(self, cmd_id, response):        try:            # Use temporary file to avoid BytesIO issues            with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:                json.dump(response, f, indent=2)                temp_path = f.name                        file_metadata = {                'name': f'result_{cmd_id}.json',                'parents': [FOLDER_ID]            }                        media = MediaFileUpload(temp_path, mimetype='application/json')                        drive_service.files().create(                body=file_metadata,                media_body=media            ).execute()                        os.unlink(temp_path)            print(f"✅ Response written: result_{cmd_id}.json")                    except Exception as e:            print(f"❌ Write response error: {e}")        def process_command(self, cmd_file):        try:            content = drive_service.files().get_media(fileId=cmd_file['id']).execute()            request = json.loads(content.decode('utf-8'))                        cmd_id = cmd_file['name'].replace('command_', '').replace('.json', '')            print(f"📋 Processing: {cmd_id}")                        result = self.execute_code(request.get('code', ''))            self.write_response(cmd_id, result)                        self.processed.add(cmd_file['id'])            self.stats['processed'] += 1                        if result['status'] == 'error':                self.stats['errors'] += 1                print(f"❌ Execution error: {result['error']}")            else:                print(f"✅ Executed successfully")                        except Exception as e:            print(f"❌ Process command error: {e}")            self.stats['errors'] += 1        def run(self, duration=1800):        self.running = True        start_time = time.time()                print(f"🚀 Processor started for {duration//60} minutes")        print("⏱️  Checking for commands every 3 seconds...")                try:            while self.running and (time.time() - start_time < duration):                commands = self.get_commands()                                if commands:                    for cmd in commands:                        self.process_command(cmd)                        if not self.running:                            break                else:                    elapsed = int(time.time() - start_time)                    print(f"⏳ No commands... running {elapsed}s (processed: {self.stats['processed']}, errors: {self.stats['errors']})")                                time.sleep(3)                        except KeyboardInterrupt:            print("🛑 Stopped by user")        except Exception as e:            print(f"❌ Main loop error: {e}")        finally:            self.running = False            print(f"🛑 Processor stopped")            print(f"📊 Final stats: {self.stats['processed']} processed, {self.stats['errors']} errors")processor = WorkingProcessor()print("✅ Working processor created and tested")

In [ ]:
# Cell 5: RUN PROCESSOR (Continuous Loop - FIXED)
print('🚀 Starting ACTIVE processor...')
print('⚠️ This cell runs continuously, NOT 0ms!')

processor.running = True
start_time = time.time()
duration = 1800  # 30 minutes

print(f'⏱️ Will run for {duration//60} minutes')
print('📊 Status updates every 5 seconds...')

try:
    while processor.running and (time.time() - start_time < duration):
        commands = processor.get_commands()
        
        if commands:
            print(f'\n📨 Found {len(commands)} commands!')
            for cmd in commands:
                processor.process_command(cmd)
                if not processor.running:
                    break
        else:
            elapsed = int(time.time() - start_time)
            print(f'⏳ Active... {elapsed}s | Processed: {processor.stats["processed"]} | Errors: {processor.stats["errors"]}')
        
        time.sleep(5)
        
except KeyboardInterrupt:
    print('🛑 Stopped by user')
    processor.running = False

print('🛑 Processor stopped')
print(f'📊 Final stats: {processor.stats}')